In [1]:
iotConnectionString = "<your IoT Hub conn string (see links at bottom)>"

In [2]:
iotEhConf = {
  'eventhubs.connectionString' : iotConnectionString,
  'eventhubs.consumerGroup' : "databricks"
}

print(iotEhConf)

In [3]:
df = spark \
  .readStream \
  .format("eventhubs") \
  .options(**iotEhConf) \
  .load()

In [4]:
data = df.selectExpr("CAST(body as STRING)")

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
jsDf = data.select(get_json_object(data['body'],"$.Id").alias('DeviceId'),
                               get_json_object(data['body'],"$.X").alias('X').cast(DoubleType()),
                               get_json_object(data['body'],"$.Y").alias('Y').cast(DoubleType()),
                               get_json_object(data['body'],"$.Z").alias('Z').cast(DoubleType()))

In [6]:
fell = jsDf.filter(jsDf['Y'] == 0.1)

In [7]:
writeConnectionString = "<your Event Hub conn string (see links at bottom)>"
ehWriteConf = {
  'eventhubs.connectionString' : writeConnectionString,
  'eventhubs.consumerGroup' : "databricks"
}

In [8]:
ds = df \
  .select("body") \
  .writeStream \
  .format("eventhubs") \
  .options(**ehWriteConf) \
  .option("checkpointLocation", "///output2.txt") \
  .start()

In [9]:
# https://docs.microsoft.com/en-us/azure/azure-databricks/databricks-stream-from-eventhubs
# https://github.com/Azure/azure-event-hubs-spark/blob/master/docs/PySpark/structured-streaming-pyspark.md